## Import các thư viện cần thiết

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors.cross_encoder_rerank import CrossEncoderReranker
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from datasets import Dataset
import pandas as pd

## Khởi chạy hệ thống RAG

In [ ]:
# Load mô hình LLM từ Ollama
def load_llm(model_name: str):
    LLM = ChatOllama(
        model = model_name,
        temperature = 0.1,
        num_predict = 256
    )
    return LLM

LLM = load_llm("vi-dominic/vinallama:7b")



# Tạo prompt cho mô hình LLM
prompt_template = """
Bạn là một trợ lý AI hữu ích của một cửa hàng cà phê Starbucks. Dưới đây là một số thông tin từ tài liệu về Starbucks. Hãy sử dụng chúng để trả lời câu hỏi của khách hàng. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời. Hãy giữ câu trả lời ngắn gọn.
Thông tin: {context}
Câu hỏi: {question}
Trả lời:
"""

def create_prompt(prompt_template):
    prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])
    return prompt

prompt = create_prompt(prompt_template=prompt_template)



# Kết nối với Vector Store
vector_db_path = r"vectorstore\faiss_db"

def load_vectorstore(model_name: str):
    embedding_model = HuggingFaceEmbeddings(model_name=model_name)
    vector_db = FAISS.load_local(folder_path=vector_db_path, embeddings=embedding_model, allow_dangerous_deserialization=True)
    return vector_db


vectorstore = load_vectorstore(model_name="hiieu/halong_embedding")

retriever = vectorstore.as_retriever()

cross_encoder_model = HuggingFaceCrossEncoder(model_name="namdp-ptit/ViRanker")
compressor = CrossEncoderReranker(model=cross_encoder_model, top_n=3)
compressor_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)


# Khai báo RAG Chain
def format_docs(documents):
    return "\n\n".join(document.page_content for document in documents)

rag_chain = (
    {"context": compressor_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | LLM
    | StrOutputParser()
)

## Load tập evaluation [ "câu hỏi" , "sự thật nền tảng" ]

In [5]:
df = pd.read_excel(r"./eval_vinallama.xlsx")
df.head(10)

,question,answer,context,ground truth
0,Gói FlavorLock™ là gì?,NaN,NaN,Công nghệ FlavorLock™ độc đáo của chúng tôi sử...
1,Tôi nên sử dụng loại cà phê xay nào?,NaN,NaN,Để có được loại cà phê có hương vị thơm ngon n...
2,Cách bảo quản cà phê tốt nhất là gì?,NaN,NaN,"Khi được rang, cà phê bắt đầu mất mùi khi tiếp..."
3,Cà phê sẽ tươi trong bao lâu?,NaN,NaN,Cà phê đựng trong túi FlavorLock™ chưa mở sẽ t...
4,Thẻ Starbucks là gì?,NaN,NaN,Thẻ Starbucks là một thẻ vật lý mang thương hi...
5,Tôi có thể nhận Thẻ Starbucks ở đâu?,NaN,NaN,Thẻ Starbucks có sẵn tại tất cả cửa hàng Starb...
6,Làm thế nào để tôi kích hoạt Thẻ Starbucks?,NaN,NaN,Thẻ Starbucks phải được kích hoạt với giá trị ...
7,Tôi có thể sử dụng Thẻ Starbucks của mình ở đâu?,NaN,NaN,Thẻ Starbucks được chấp nhận tại tất cả cửa hà...
8,Tôi có thể sử dụng Thẻ Starbucks của mình khi ...,NaN,NaN,Thẻ Starbucks được kích hoạt tại Việt Nam chỉ ...
9,Làm thế nào để tôi đăng ký Thẻ Starbucks?,NaN,NaN,Bạn có thể đăng ký Thẻ Starbucks một cách dễ d...


In [6]:
# Biến đổi câu hỏi và sự thật nền tảng thành các mảng
questions = df["question"].to_list()
ground_truth = df["ground truth"].to_list()

In [7]:
# Các câu hỏi
questions[:10]

['Gói FlavorLock™ là gì?',
 'Tôi nên sử dụng loại cà phê xay nào?',
 'Cách bảo quản cà phê tốt nhất là gì? ',
 'Cà phê sẽ tươi trong bao lâu?',
 'Thẻ Starbucks là gì?',
 'Tôi có thể nhận Thẻ Starbucks ở đâu?',
 'Làm thế nào để tôi kích hoạt Thẻ Starbucks?',
 'Tôi có thể sử dụng Thẻ Starbucks của mình ở đâu?',
 'Tôi có thể sử dụng Thẻ Starbucks của mình khi tôi đi du lịch ở nước ngoài được không?',
 'Làm thế nào để tôi đăng ký Thẻ Starbucks?']

In [8]:
# Các sự thật nền tảng
ground_truth[:10]

['Công nghệ FlavorLock™ độc đáo của chúng tôi sử dụng loại van một chiều đặc biệt cho phép carbon dioxide từ các hạt cà phê rang tươi giải phóng khỏi gói cà phê đã dán kín mà không cho khí ôxy làm nhạt hương vị lọt vào. Tất cả các loại cà phê Starbucks được bán trong gói nửa pound (225g) và một pound (450g) được dán kín trong các gói FlavorLock™ kín khí trong 2 giờ kể từ khi rang, nhờ đó nó giữ được mùi vị thơm ngon cho đến khi bạn sẵn sàng sử dụng.',
 'Để có được loại cà phê có hương vị thơm ngon nhất, chúng tôi tiếp tục khuyên bạn nên bắt đầu với cà phê nguyên hạt và xay mới cho từng ấm. Hoặc nếu thích, bạn có thể đem cà phê nguyên hạt tới cửa hàng bán lẻ tại địa phương và nhờ họ xay cho bạn.',
 'Khi được rang, cà phê bắt đầu mất mùi khi tiếp xúc với không khí và độ ẩm lâu hơn. Chúng tôi khuyên bạn nên mua cà phê hàng tuần, bảo quản trong thùng chứa kín ở nơi mát mẻ, ít ánh sáng và chỉ xay cà phê trước khi pha. ',
 'Cà phê đựng trong túi FlavorLock™ chưa mở sẽ tươi trong vài tháng.',

## Sử dụng RAG Chain để trả lời các câu hỏi trong tập evaluation

In [ ]:
data  = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

# Trả lời tất cả câu hỏi và lưu lại câu trả lời, ngữ cảnh
for query in questions:
    data["question"].append(query)
    data["answer"].append(rag_chain.invoke(query))
    data["contexts"].append([doc.page_content for doc in compressor_retriever.invoke(query)])

eval_dataset = Dataset.from_dict(data)

In [ ]:
# Lưu tập đã trả lời thành một file csv
csv_df = pd.DataFrame.from_dict(data)
csv_df.to_csv("./eval_vinallama-7b.csv", index=False)

In [ ]:
# Đọc file csv vừa được lưu
result_csv = pd.read_csv("./eval_vinallama-7b.csv")
result_csv.head(10)

,question,answer,contexts,ground_truth
0,Gói FlavorLock™ là gì?,Gói FlavorLock™ là một gói kín khí được sử dụn...,['Gói FlavorLock™ là gì?\nCông nghệ FlavorLock...,Công nghệ FlavorLock™ độc đáo của chúng tôi sử...
1,Tôi nên sử dụng loại cà phê xay nào?,Bắt đầu với cà phê nguyên hạt và xay mới cho m...,['Tôi nên sử dụng loại cà phê xay nào?\nĐể có ...,Để có được loại cà phê có hương vị thơm ngon n...
2,Cách bảo quản cà phê tốt nhất là gì?,"Mua cà phê hàng tuần, bảo quản trong thùng chứ...",['Câu hỏi thường gặp về Chuẩn bị Cà phê\nCách ...,"Khi được rang, cà phê bắt đầu mất mùi khi tiếp..."
3,Cà phê sẽ tươi trong bao lâu?,Cà phê đựng trong túi FlavorLock™ chưa mở sẽ t...,['Câu hỏi thường gặp về Chuẩn bị Cà phê\nCách ...,Cà phê đựng trong túi FlavorLock™ chưa mở sẽ t...
4,Thẻ Starbucks là gì?,Thẻ Starbucks là một thẻ nhựa vật lý mang thươ...,['Thẻ Starbucks là một thẻ nhựa vật lý mang th...,Thẻ Starbucks là một thẻ vật lý mang thương hi...
5,Tôi có thể nhận Thẻ Starbucks ở đâu?,Thẻ Starbucks có sẵn tại tất cả cửa hàng Starb...,['Câu hỏi thường gặp chương trình Starbucks Re...,Thẻ Starbucks có sẵn tại tất cả cửa hàng Starb...
6,Làm thế nào để tôi kích hoạt Thẻ Starbucks?,Thẻ Starbucks phải được kích hoạt với giá trị ...,['Câu hỏi thường gặp chương trình Starbucks Re...,Thẻ Starbucks phải được kích hoạt với giá trị ...
7,Tôi có thể sử dụng Thẻ Starbucks của mình ở đâu?,Thẻ Starbucks được chấp nhận tại tất cả cửa hà...,['Thẻ Starbucks có sẵn tại tất cả cửa hàng Sta...,Thẻ Starbucks được chấp nhận tại tất cả cửa hà...
8,Tôi có thể sử dụng Thẻ Starbucks của mình khi ...,Thẻ Starbucks được kích hoạt tại Việt Nam chỉ ...,['Thẻ Starbucks có sẵn tại tất cả cửa hàng Sta...,Thẻ Starbucks được kích hoạt tại Việt Nam chỉ ...
9,Làm thế nào để tôi đăng ký Thẻ Starbucks?,Bạn có thể đăng ký Thẻ Starbucks một cách dễ d...,['Làm thế nào để tôi đăng ký Thẻ Starbucks?\nB...,Bạn có thể đăng ký Thẻ Starbucks một cách dễ d...
